# Deploy a Serverless Model Server with Nuclio-KFServing
  --------------------------------------------------------------------

The following notebook demonstrates how to deploy **any pickled model** using **[nuclio](https://github.com/nuclio/nuclio)** + **[KFServing](https://github.com/kubeflow/kfserving)** (a.k.a <b>Nuclio-serving</b>)

#### **notebook how-to's**
* Write and test model serving (KFServing) class in a notebook.
* Deploy the model server as a Nuclio-serving function.
* Invoke and test the serving function.

<a id='top'></a>
#### **steps**
**[define a new function and its dependencies](#define-function)**<br>
**[test the model serving class locally](#test-locally)**<br>
**[deploy our serving class using as a serverless function](#deploy)**<br>
**[test our model server using HTTP request](#test-model-server)**<br>

In [1]:
# nuclio: ignore
# if the nuclio-jupyter package is not installed run !pip install nuclio-jupyter
import nuclio

<a id='define-function'></a>
### **define a new function and its dependencies**

In [2]:
%nuclio config kind="nuclio:serving"
%nuclio env MODEL_CLASS=ClassifierModel

%nuclio: setting kind to 'nuclio:serving'
%nuclio: setting 'MODEL_CLASS' environment variable


In [3]:
%%nuclio cmd -c
pip install -U -q kfserving
pip install -U -q azure
pip install -U -q mlrun

In [4]:
#%nuclio config spec.build.image = "yjbds/mlrun-serving:dev"

In [5]:
import kfserving
import os
import numpy as np
from cloudpickle import load

In [6]:
TARGET_PATH = '/User/repos/demos/dask/artifacts'
MODEL_FILE = 'lgbm-model.pkl'

In [7]:
class ClassifierModel(kfserving.KFModel):
    def __init__(self, name: str, model_dir: str, model = None):
        super().__init__(name)
        self.name = name
        self.model_dir = model_dir
        if not model is None:
            self.classifier = model
            self.ready = True

    def load(self):
        model_file = os.path.join(
            kfserving.Storage.download(self.model_dir), MODEL_FILE)
        self.classifier = load(open(model_file, 'rb'))
        self.ready = True

    def predict(self, body):
        try:
            feats = np.asarray(body['instances'])
            result: np.ndarray = self.classifier.predict(feats)
            return result.tolist()
        except Exception as e:
            raise Exception("Failed to predict %s" % e)

The following end-code annotation tells ```nuclio``` to stop parsing the notebook from this cell. _**Please do not remove this cell**_:

In [8]:
# nuclio: end-code

______________________________________________

<a id='test-locally'></a>
### **test the model serving class locally**
The class above can be tested locally. Just instantiate the class, `.load()` will load the model to a local dir.

> **Verify there is a `model.bst` file in the model_dir path (generated by the training notebook)**

In [9]:
model = load(open(TARGET_PATH + '/' + MODEL_FILE, 'rb'))

/User/.pythonlibs/jupyter-1/lib/python3.6/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.preprocessing.label module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.preprocessing. Anything that cannot be imported from sklearn.preprocessing is now part of the private API.
  warnings.warn(message, FutureWarning)
/User/.pythonlibs/jupyter-1/lib/python3.6/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator LabelEncoder from version 0.21.3 when using version 0.22.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [10]:
model

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1,
               local_listen_port=12400,
               machines='10.233.64.59:12400,10.233.64.53:12401,10.233.64.54:12402,10.233.64.56:12403,10.233.64.58:12404,10.233.64.55:12405,10.233.64.57:12406,10.233.64.52:12407',
               max_depth=3, min_child_samples=20, min_child_weight=0.001,
               min_split_gain=0.0, n_estimators=3, n_jobs=-1, num_leaves=31,
               num_machines=8, num_threads=1, objective=None, random_state=1,
               reg_alpha=0.0, reg_lambda=0.0, silent=False, subsample=1.0,
               subsample_for_bin=200000, subsample_freq=0, time_out=120,
               tree_learner='data')

In [11]:
my_server = ClassifierModel('classifier', model_dir=TARGET_PATH, model = model)
my_server.load()

[I 200216 14:56:42 storage:35] Copying contents of /User/repos/demos/dask/artifacts to local


### _data_
Make some classification data using scikit learn's `make_classification`:

In [12]:
# import dask
# import pandas as pd
# import dask.dataframe as dd
# import pyarrow.parquet as pq
# import pyarrow

# xtest = pd.read_parquet('/User/repos/demos/dask/artifacts/test_set')

# xtest.head()

# xtest.pop('index')

# ytest = xtest.pop('ArrDelay')

# event = {"instances": xtest.values.tolist()}

# ytest

# my_server.predict(event)

<a id='deploy'></a>
### **deploy our serving class using as a serverless function**

In [13]:
import mlrun

In [14]:
fn = mlrun.new_model_server('generic', 
                            models={'classifier_gen': TARGET_PATH}, 
                            model_class='ClassifierModel').apply(mlrun.mount_v3io())

In [15]:
addr = fn.deploy()

[mlrun] 2020-02-16 14:56:59,377 deploy started
[nuclio] 2020-02-16 15:01:00,676 (info) Build complete
[nuclio] 2020-02-16 15:02:02,692 (error) Failed to deploy. Details:
[E 200216 15:01:42 logger:106] Caught unhandled exception while initializing
[E 200216 15:01:42 logger:106] Caught unhandled exception while initializing
[E 200216 15:01:42 logger:106] Caught unhandled exception while initializing
[E 200216 15:01:42 logger:106] Caught unhandled exception while initializing
[E 200216 15:01:42 logger:106] Caught unhandled exception while initializing
[E 200216 15:01:42 logger:106] Caught unhandled exception while initializing
Caught unhandled exception while initializing [err="'str' object has no attribute 'name'" || traceback="Traceback (most recent call last):
  File "/opt/nuclio/_nuclio_wrapper.py", line 272, in run_wrapper
    args.trigger_name)
  File "/opt/nuclio/_nuclio_wrapper.py", line 56, in __init__
    self._context = nuclio_sdk.Context(self._logger, self._platform, worker_id

DeployError: cannot deploy 

<a id="test-model-server"></a>
### **test our model server using HTTP request**

In [ ]:
import json
import requests

resp = requests.post(addr + '/classifier_gen/predict', json=event)

In [ ]:
resp.__dict__['_content'] 

In [ ]:
json.loads(resp.content)

**[back to top](#top)**